# Image Classification Test Model

In [ ]:
#---- Install stuff -------
#!pip install --upgrade pip

#!pip install opencv-python
#!pip install tensorflow
#!pip install numpy==1.19.5

#!pip install Pillow
#!pip install playsound
#!pip install gTTS

#!pip install matplotlib
#!pip install pandas
#!pip install seaborn

#!pip install PyYAML

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
import random
from glob import glob

In [ ]:
import cv2
from PIL import Image

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

In [ ]:
from gtts import gTTS
from playsound import playsound
import IPython
from IPython.core.display import display

def text2audio(mytext):    
    myobj = gTTS(text=mytext, lang='en', slow=False)
    myobj.save("./tts.mp3")    
    display(IPython.display.Audio("./tts.mp3", autoplay=True))

In [ ]:
#print system info
import sys
print('Python: ',sys.version)

In [ ]:
#print tensor and keras version
print('keras: ', keras.__version__)
print('tensorflow: ', tf.__version__)
import numpy
print('numpy: ', numpy.version.version)

# Read Cfg

In [ ]:
#cfg_file = './project/work_pose.yml'
cfg_file = './project/home_presence.yml'

#read cfg
import yaml

with open(cfg_file, "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.CLoader)

In [ ]:
project_name = cfg["project_name"]
print('project_name: ',project_name)
print('-'*20)

dir = cfg["dir"]
temp_dir = cfg["temp_dir"]
print('project_dir: ',dir)
print('temp_dir: ',temp_dir)
print('-'*20)
#read from cfg file.. since url can have pwd
src_video = cfg["src_video"]

labels = cfg['labels'].split(' ')
print('labels: ',labels)
print('label count: ',len(labels))
print('-'*20)
#divide width and height of image by n (reduce resolution)
reduce_image_wh_by = cfg['reduce_image_wh_by']
print('reduce_image_wh_by: ',reduce_image_wh_by)

crop_image_from_left = cfg['crop_image_from_left']
crop_image_from_right = cfg['crop_image_from_right']
print('crop_image_from_left: ',crop_image_from_left)
print('crop_image_from_right: ',crop_image_from_right)
print('-'*20)
model_file = cfg["model_file"]
print('model_file: ',model_file)

In [ ]:
project_dir = dir + project_name + "/"
project_temp_dir = temp_dir + project_name + "/"
print('project_dir: ',project_dir)
print('project_temp_dir:', project_temp_dir)

## Functions

In [ ]:
def get_capture_dim():
    cap = None
    try:
        cap = cv2.VideoCapture(src_video)
        if(cap.isOpened()):
            ret, frame = cap.read()
            return frame.shape
        else:
            return 0,0
    finally:
        if cap!=None:
            cap.release()
        cv2.destroyAllWindows()

In [ ]:
#org shape 1080, 1920
capture_dim = get_capture_dim()
print('org dim: ',capture_dim)

img_h = capture_dim[0]
img_w = capture_dim[1]

img_h = img_h//reduce_image_wh_by
img_w = img_w//reduce_image_wh_by

print('new dim h:',img_h)
print('new dim w:',img_w)

In [ ]:
def speak(mytext):
    myobj = gTTS(text=mytext, lang='en', slow=False)
    myobj.save("./speak.mp3")
    playsound("./speak.mp3")

In [ ]:
def plot_image(images, captions=None, cmap=None):    
    if captions!=None:
        print(captions)
    
    if len(images) > 1:
        f, axes = plt.subplots(1, len(images), sharey=True, figsize=(4,4))
        f.set_figwidth(15)
        for ax,image in zip(axes, images):
            ax.imshow(image, cmap)
    else:
        plt.figure(figsize=(4,4))
        plt.imshow(images[0])
    plt.show()

In [ ]:
def get_file_name():
    return 'img_'+datetime.now().strftime("%Y%m%d%H%M%S")
#get_file_name()

In [ ]:
def capture_frames(h,w, crop=True, count=1, delay_sec=60, save=True):
    cap = None
    if h==0 or w==0:
        raise Exception('h or w can not be 0')
    try:
        cap = cv2.VideoCapture(src_video)
        #cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        
        if(cap.isOpened()):
            for i in range(0, count):
                ret, frame = cap.read()
                if not ret:
                    print("failed to grab frame ")
                    #raise Exception("failed to grab frame ")
                    return None, None

                #print('org shape: ',frame.shape)
                frame = cv2.resize(frame, (w,h), interpolation = cv2.INTER_AREA)

                if crop:
                    if crop_image_from_left>0:
                        new_wl = int(w*crop_image_from_left)
                    else:
                        new_wl = 0
                        
                    if crop_image_from_right>0:
                        #crop 70% on width from right
                        new_wr = int(w*crop_image_from_right)
                    else:
                        new_wr = w                    
                        
                    frame = frame[0:h,new_wl:new_wr]

                if save:
                    img_name = loc_unknown+"frame_{0}.png".format(get_file_name())
                    cv2.imwrite(img_name, frame)
                else:
                    img_name = None

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                if count>1 and save:
                    print('file saved: ',img_name)
                    
                if (i+1)!=count:
                    #close and re-open else we will get old frame
                    if cap != None:
                        cap.release()
                    #it take about 2 sec to open cam again
                    if delay_sec>0:
                        time.sleep(delay_sec-2) 
                    cap = cv2.VideoCapture(src_video)
                    if(cap.isOpened()==False):
                        print('Could not open camera!')
                        break            
            return frame, img_name
        else:
            print('Could not open camera!')
            return None, None
    finally:
        if cap != None:
            cap.release()
        cv2.destroyAllWindows()

# Load Model

In [ ]:
class RandomColorDistortion(tf.keras.layers.Layer):
    contrast_range=[-1.0, 1.0]
    brightness_delta=[-50, 50]

    def __init__(self, **kwargs):
        super(RandomColorDistortion, self).__init__(**kwargs)

    def call(self, images, training=True):
        if not training:
            return images

        contrast = np.random.uniform(self.contrast_range[0], self.contrast_range[1])
        brightness = np.random.uniform(self.brightness_delta[0], self.brightness_delta[1])

        #print('brightness: ',brightness, ', contrast: ',contrast)

        #images = tf.image.adjust_contrast(images, contrast)
        images = tf.image.adjust_brightness(images, brightness)
        images = tf.clip_by_value(images, 0, 255)
        return images

In [ ]:
num_classes = len(labels)

from tensorflow.keras.utils import CustomObjectScope

print('model_file: ',model_file)

with CustomObjectScope({'RandomColorDistortion': RandomColorDistortion}):
    model = tf.keras.models.load_model(model_file)
    
print (model.summary())

# Load Test Image

In [ ]:
print('project_dir: ',project_dir)

In [ ]:
all_images = glob(project_dir+'*/*.png')
print('all img count: ',len(all_images))

In [ ]:
from numpy import asarray

#file = random.choice(all_images)
frame, file = capture_frames(img_h,img_w, crop=True, count=1, delay_sec=0, save=False)

if False:
    frame = Image.open(file)
    frame = asarray(frame)
    
if True:
    if file!=None:
        print('file: ', file)
        frame = keras.preprocessing.image.load_img(file, target_size=(img_h,img_w))
    frame = asarray(frame)
    #frame = keras.preprocessing.image.img_to_array(frame)
    
plot_image([frame])
#print('type:',type(frame))
#print('shape:',frame.shape)
#print('-')

img_array = tf.expand_dims(frame, 0) # Create a batch
#img_array = frame.reshape(1, 360,448,3)
#print('shape:',img_array.shape)
#print('-')

predictions = model.predict(img_array)
print('predictions:',predictions)
score = tf.nn.softmax(predictions[0])
print('score:',score)
print('-')
print('Label:',labels[np.argmax(score)],'[', round(100*np.max(score),3), '%]')
print('-'*25)
for l,s in zip(labels,list(score*100)):
    print(l.rjust(10,' '),round(float(s),3))